# Map Business Terms to Data Headers

## Introduction

In this notebook we programmatically publish a dataset into a catalog and map business terms to the dataset column headers. The business terms and their mappings are specified in a csv file included with the project. The user must first ensure that the catalog exists and the imported business terms have been published.

The user can also assign business terms to column headers manually or by using the Data Discovery capability within Cloud Pak for Data. 

This notebook is optional. The analytics project runs as expected even if this notebook is not used. 

**Note that as only Admin users can import terms, this notebook should be run by an Admin user only.**

In [2]:
# imports for the rest APIs interactions with WKC
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
import json
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from pandas.io.json import json_normalize
import pandas as pd
import os
# use this library for reading and saving data in CP4D
from project_lib import Project
project = Project()

**This project contains Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.<br>**

## Create Catalog

The dataset must first be published into a catalog. The catalog must be manually created. Under **Organize** in the navigation menu, select **All Catalogs** and select **New Catalog**. Enter the name for the catalog and the description if necessary and create the catalog. If the user has already created the catalog this step can be skipped and the existing catalog name should be specified in the code cell below.


## User Inputs

The user must enter the following before running the rest of the notebook: 
1. **host :** host url of the cluster we are working on.
2. **uname :** username for user on this cluster.
3. **pword :** password for user on this cluster.
4. **catalog_name :** Name of the catalog that we would like to publish the csv to. This catalog is created based on the instructions above or an existing catalog.

In [1]:
# sample input and syntax
# host = 'https://xxxxx.com'
# uname = 'admin'
# pword = '******'
# catalog_name = 'name for the catalog'

host = '******************'
uname = '**********'
pword = '***********'
catalog_name = '**********'

We also create additional variables. The user does not need to change the code cell below, unless they change the business terms category name or the name of the csv file with mappings.

1. **category_name :** Name of the business term category corresponding to the project.
2. **terms_file :** Name of the csv file containing the list of mappings between column headers and business terms.
3. **csv_file_to_publish :** Name of the csv files that will be published into the catalog and for which we map business terms.

In [3]:
category_name = "Utilities Customer Attrition Prediction"
terms_file = "utilities customer attrition prediction map terms.csv" 
csv_file_to_publish = 'Attrition View.csv'

Create a requests session and use the same session throughout the notebook. 

In [4]:
# Creates requests session and stores in `s`
s = requests.Session()

## Authentication

Generate a token and validate the token on this cluster.

In [5]:
# Authenticate the cluster with specified username and password and store the access token for future reference
wkcURLauth=host+"/icp4d-api/v1/authorize"

# Payload with username and password
payload={
    "username": uname,
    "password": pword
}

# Header with json format
headers = {
    'Content-Type': "application/json",
    'cache-control': "no-cache"
    }

# Creates a post request with the endpoints specified above in the wkcURLauth variable with payload and header.
# When successfully authenticated token is stored in a variable as below
# catch error if the specified url is not correct
try:
    res = s.post(wkcURLauth,headers=headers,json=payload, verify=False)
    
except:
    print("The below error has occurred. Please check that the hostname entered is correct.")
    raise
    
if res.status_code == 200:
    print("Authentication Successful")
    accessToken=json.loads(res.text)['token']
else:
    print('The below error has occurred. Please check entered username and password are correct.')
    raise ValueError(res.text)

Authentication Successful


## Map Business Terms to Headers

We complete the following steps to map the business terms to column headers:

1. Check if the Category `Utilities Customer Attrition Prediction` exists in the parent category `Industry Accelerators`.
2. Load the business terms from the `Utilities Customer Attrition Prediction` subcategory into a dataframe.
3. Publish the specified dataset into the catalog.
4. Assign business terms to the dataset column headers.

### 1. Check for the Category

Below cell fetches all the categories present in the cluster and stores category id of the category `Utilities Customer Attrition Prediction`.

In [6]:
search_url=host+"/v3/search"
try:
    headers = {
        'Content-Type': "application/json",
        'Authorization': "Bearer "+accessToken,
        'Cache-Control': "no-cache",
        'Connection': "keep-alive"
        }
    
    search_body = {
        "size": 1000,
        "_source": ["artifact_id","metadata.name"],
       "query": {    
               "match": {"metadata.artifact_type": "category"}
       }
    }
    parent_cat = s.post(search_url, verify=False,  json=search_body, headers=headers)
    
    
    
    # Check if Industry accelerator category exists and load its id into a variable `parent_id`
    if parent_cat.status_code == 200:
        category=json.loads(parent_cat.text)
        for i in category['rows']:
            
            if i['metadata']['name']== category_name:
                print("Category ",category_name,"exists")

                exists_category=True
                category_id=i['artifact_id'] 
            
                   
except:
    print("The below error has occurred. " + "Please ensure that category, '" + category_name + "', exists.")
    raise ValueError(parent_cat.text)

Category  Utilities Customer Attrition Prediction exists


### 2. Load category Business Terms into Dataframe 

Get all of the terms in the `Utilities Customer Attrition Prediction` category and store them in the `df_terms` dataframe.

In [8]:
# Create a payload for the post request, This payload contains information on size of the terms, source, category and subcategory ids
payload={"size":300,"from":0,"_source":["artifact_id","metadata.artifact_type","metadata.name","metadata.description","categories","entity.artifacts"],"query":{"bool":{"filter":{"bool":{"minimum_should_match":1,"should":[{"term":{"categories.primary_category_id":category_id}},{"term":{"categories.secondary_category_ids":category_id}}],"must_not":{"terms":{"metadata.artifact_type":["category"]}}}}}}}
# create a post request with above payload 
wf=s.post(host+"/v3/search",headers=headers,json=payload,verify=False)
# it will return all the terms , load these terms into a dataframe
wf_json=json.loads(wf.text)['rows']
df_terms=pd.json_normalize(wf_json)

df_terms=df_terms[['entity.artifacts.global_id','metadata.name']]

In [9]:
# terms dataframe looks as below
df_terms.head()

,entity.artifacts.global_id,metadata.name
0,5d2d5419-0032-4c64-90e2-ce68c6997bb5_a7b6b7bd-...,Communication Resolution Type
1,5d2d5419-0032-4c64-90e2-ce68c6997bb5_30f2edd4-...,Customer Emails
2,5d2d5419-0032-4c64-90e2-ce68c6997bb5_b3445679-...,Customer Agreement Number
3,5d2d5419-0032-4c64-90e2-ce68c6997bb5_0828700d-...,Customer Attrition Probability
4,5d2d5419-0032-4c64-90e2-ce68c6997bb5_b288dc0a-...,Electronic Bill Indicator


### 3. Publish Dataset into Catalog

Get the ID of the catalog that was specified in the user inputs at the beginning of this notebook.

In [10]:
## Get catalog that created and its id by providing name of the catalog created, wich should be same as the one entered in the previous cells
catalog_endpoint=host+"/v2/catalogs"
# Create new header for the requests
headers = {
'Content-Type': "application/json",
'Authorization': "Bearer "+accessToken

}

# endpoint to get all the catalogs 
get_catalog=s.get(catalog_endpoint,verify=False, headers=headers)


## Find the catalog created with specific name and store name and id of it into catalog_name and catalog_id respectively
try:
    get_catalog_json=json.loads(get_catalog.text)['catalogs']
except:
    print("The below error has occurred. Please ensure that catalog, '" + catalog_name + "', exists")
    raise
    
catalog_id = ''
for metadata in get_catalog_json:
    if metadata['entity']['name']==catalog_name:
        catalog_id=metadata['metadata']['guid']
        print("catalog_id for",catalog_name, catalog_id)

if catalog_id == '':
    print("The provided catalog name cannot be found. Please ensure that catalog, '" + catalog_name + "', exists")
    raise ValueError("Catalog cannot be found")

catalog_id for vb_catalog 9b66abed-b564-4f86-bc3b-f689a4808994


Get the project id. All project assets can be accessed using this project id.

In [11]:
project_id=os.environ['PROJECT_ID']

Get all existing csv files in the project folder and store the names of these files. 

In [12]:
# payload 
payload={"query":"*:*","limit":200}
# endpoint to access all the project assets in the project folder 
asset_url=host+"/v2/asset_types/asset/search?project_id="+project_id
get_asset=s.post(asset_url,json=payload,verify=False)

Next we get the asset id of the dataset to be published to the catalog.

In [13]:
# Get asset ids of all csv files to be published in to the catalog and store the asset ids in an array

project_asset_id=[]
# Payload to query all project assets
payload={"query":"*:*","limit":200}

get_asset=s.post(host+"/v2/asset_types/asset/search?project_id="+project_id,json=payload,verify=False, headers=headers)
get_asset_json=json.loads(get_asset.text)
for j in get_asset_json['results']:
    if j['metadata']['name']==csv_file_to_publish:
        print("Asset id of",csv_file_to_publish,":",j['metadata']['asset_id'])
        project_asset_id.append(j['metadata']['asset_id'])

Asset id of Attrition View.csv : 7c520314-a5dd-4702-aff4-0a4ff586329e


Using the asset ID for the dataset, upload the dataset into the catalog using the post request below. Get the new asset ID of the newly published dataset.

In [14]:
print("ASSET ID's of the published assets")
# Creates a empty dictionary
catalog_asset_ids={}
for asset_id in project_asset_id:
    #for  each asset in the project , publish them into the catalog 
    # pyload to publish the asset
    payload={"mode":0,"catalog_id":catalog_id,"metadata":{}}
    # endpoint to publish asset
    asset_publish_url=host+"/v2/assets/"+asset_id+"/publish?project_id="+project_id
    # Post request with endpoint, heaeder and payload
    publishasset=requests.post(asset_publish_url,json=payload,headers=headers,verify=False)
    # api endpoint returns below text
    publishasset_json=json.loads(publishasset.text)
    # extract csv file published and its asset id and append it to the dictionary
    catalog_asset_ids[publishasset_json['metadata']['name']]=publishasset_json['asset_id']
    
print(catalog_asset_ids)

ASSET ID's of the published assets
{'Attrition View.csv': '53e67e75-fc43-4de9-9dc6-44e9f177d71e'}


### 4. Assign Business Terms to Column Headers

Read in the file with business terms and their associated column headers and view a sample of the data.

In [15]:
my_file = project.get_file(terms_file)
my_file.seek(0)
map_terms = pd.read_csv(my_file)

In [16]:
# comment out below set of codes if the map terms csv has additional spaces which need to be truncated.
#map_terms['Column_header']=map_terms['Column_header'].str.strip(' ')
#map_terms["Business Terms"]=map_terms["Business Terms"].str.strip(' ')
#map_terms["File"]=map_terms["File"].str.strip(' ')
#map_terms["Table"]=map_terms["Table"].str.strip(' ')

print(map_terms.shape)
map_terms.head()

(65, 4)


,Business Terms,Column_header,Table,File
0,Account Number,ACCOUNT_ID,ENERGY and UTILITIES ACCELERATORS,Attrition View
1,Address Line 1 Description,APPT_NBR,ENERGY and UTILITIES ACCELERATORS,Attrition View
2,Car Ownership Indicator,IS_CAR_OWNER,ENERGY and UTILITIES ACCELERATORS,Attrition View
3,City Name,CITY,ENERGY and UTILITIES ACCELERATORS,Attrition View
4,Communication Type,ISSUE_CD,ENERGY and UTILITIES ACCELERATORS,Attrition View


In [17]:
df_terms.head()

,entity.artifacts.global_id,metadata.name
0,5d2d5419-0032-4c64-90e2-ce68c6997bb5_a7b6b7bd-...,Communication Resolution Type
1,5d2d5419-0032-4c64-90e2-ce68c6997bb5_30f2edd4-...,Customer Emails
2,5d2d5419-0032-4c64-90e2-ce68c6997bb5_b3445679-...,Customer Agreement Number
3,5d2d5419-0032-4c64-90e2-ce68c6997bb5_0828700d-...,Customer Attrition Probability
4,5d2d5419-0032-4c64-90e2-ce68c6997bb5_b288dc0a-...,Electronic Bill Indicator


Join the `df_terms` and `map_terms` dataframes and map each column header to a business term. The code below loops through each file in the catalog (one file in our case) and performs the following tasks:

1. Create a dataframe with column headers in the catalog and associated business term and term ids.
2. Fetch catalog asset id for each csv in the catalog.
3. Create a column_info attribute for all the files in the catalog.
4. Map column header to the business terms. 

In [18]:
# For every file in the map terms csv do the following
# Join the csv with specified file name with the published terms to get its term id
# drop if any duplicates found to avoid multiple mappings for the same term

#map_terms=map_terms[map_terms['File']==file]
map_terms=map_terms.sort_values(by=['File','Column_header'])
Terms_Headers=pd.merge(map_terms,df_terms,left_on='Business Terms',right_on='metadata.name',how='inner')
Terms_Headers=Terms_Headers.drop_duplicates()

for file in catalog_asset_ids:#map_terms.File.unique():
    # Catalog asset id of the particular csvs
    # for each file name in the map_terms if the csv with this file name exists, get its asset_id from the catalog and use the post request publish create column_info attribute
    # This column info attribute is necessary to map the busines terms to column to header
    

    catalog_asset_id=catalog_asset_ids[file]
    print(file,  catalog_asset_id)
    #### 
    payload={"name": "column_info",
       "entity":{
                  #"sample_size":50
               }
    }
    t=requests.post(host+"/v2/assets/"+catalog_asset_id+"/attributes?catalog_id="+catalog_id,json=payload,headers=headers,verify=False)
    #print(t.text)
    # For each column header in the file map its corresponding business term retrieved from the above join in the dataframe

    i=0
    for index, rows in Terms_Headers.iterrows(): 
        i+=1
        print(i,rows.Column_header.strip(), "is mapped to", rows['Business Terms'])
        # Create list for the current row 
        # Below payload is used for the patch request to map the  header to business terms
        payload=[{"op":"add","path":"/"+rows.Column_header.strip(),"value":{"column_terms":[{"term_display_name":rows['Business Terms'],"term_id":rows["entity.artifacts.global_id"]}]},"attribute":"column_info"}]
    #
        # Endpoint for patch request
        url=host+"/v2/assets/"+catalog_asset_id+"/attributes/column_info?catalog_id="+catalog_id
    # patch request to map busines terms to column header using term_id
        patch_attribute=s.patch(url,json=payload,headers=headers,verify=False)
    #
        json.loads(patch_attribute.text) 

Attrition View.csv 53e67e75-fc43-4de9-9dc6-44e9f177d71e
1 ACCOUNT_ID is mapped to Account Number
2 ADDRESS is mapped to Postal Address
3 AGE is mapped to Date Of Birth
4 DATE_OF_BIRTH is mapped to Date Of Birth
5 APPT_NBR is mapped to Address Line 1 Description
6 ATTRITION_STATUS is mapped to Customer Attrition Rate
7 CALL_CENTER_RESPONSE is mapped to Customer Communication
8 EMAIL_RESPONSE is mapped to Customer Communication
9 SMART_METER_COMMENTS is mapped to Customer Communication
10 CITY is mapped to City Name
11 CLTV is mapped to Customer Lifetime Value
12 CLTV_RATIO is mapped to Customer Lifetime Value
13 CLTV_RATIO is mapped to Customer Acquisition Cost
14 COMPLAINTS is mapped to Customer Complaints
15 ISSUE is mapped to Customer Complaints
16 ISSUE_CD is mapped to Customer Complaints
17 ISSUE_ID is mapped to Customer Complaints
18 CREDIT_RATING is mapped to Customer Credit Authority Level
19 CST_SEGMENT_ID is mapped to Customer Market Segment
20 CURRENT_CONTRACT_ID is mapped to

The specified dataset is now published to the catalog and its column headers are mapped to their associated business terms. 

Navigate to below path to verify the mappings created above. <br>

**All Catalogs --> New Catalog --> csv file --> any column header from the above list**.

The associated business term for the column header is displayed.

In [19]:
s.close()